In [56]:
import os
import random as rnd
import pandas as pd 

import pgutils as pg
import pgmath as pm
import algorithm as al
import numpy as np

In [57]:
root_dir = r'C:\Users\eleon\PycharmProjects\SM_Exam\data\yes_small'
#root_dir = './data/yes_small'

In [58]:
#test_data=[]
with open(os.path.join(root_dir, 'test.txt')) as f:
#with open(root_dir + "\\test.txt", "r") as f:

    test_data = f.readlines()
    
test_dataset = pg.data_to_list(test_data[2:])

In [59]:
#train_data=[]
with open(os.path.join(root_dir,"train.txt"), "r") as f:
    train_data = f.readlines()
    
#train_dataset = pg.data_to_list(train_data[2:])

In [60]:
song_hash = pd.read_csv(os.path.join(root_dir,"song_hash.txt"), sep="\t", header=None)


In [61]:
# setting parameters
dimension = 2#rnd.choice([2, 5,10,25,50,100])
# regularization parameter set by cross validation
lam = rnd.choice([0.0001, 0.001, 0.01, 0.1, 1, 10, 20, 50, 100, 500, 1000])
# regularization parameter for dual point model
nu = rnd.choice([0, lam])
# threshold for landmark dimension
r = 0.2
# number of landmark
n_landmarks = 3#50
# num iteration 100 o 200
n_iter = 60 #rnd.choice([100, 200])
# tau predefined learning rate
tau = 0.5

In [62]:
toy_dataset=[[0,6,3,2,1,4,2],[2,0,6,5,6,3,0],[0,5,2,1,6],[5,6,2],[6,5,2,3,5,6,2]]

songs = 7
transition_matrix = pg.transition_count(songs, toy_dataset)
position = np.random.rand(songs, dimension)
num_transition = np.sum(transition_matrix)
params = pm.AlgParams(lam, nu, tau, num_transition, n_landmarks, r, dimension, n_iter)

In [63]:
X = al.single_point_algorithm(songs, transition_matrix, params)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59


In [64]:
dummy_landmarks = [[i for i in range(songs)] for j in range(songs)]
d2 = pm.Distances.delta(X)
prob_matrix = np.exp(-d2)/pm.Distances.zeta(d2,X,dummy_landmarks)
cum_sum = np.sum(prob_matrix, axis=1)
prob_matrix = prob_matrix / cum_sum

In [65]:
print(prob_matrix)
print(np.sum(prob_matrix, axis=1))

[[6.54396929e-01 1.38094947e-01 7.08850050e-02 8.85687416e-03
  1.00209450e-01 1.61645422e-04 3.32458522e-02]
 [1.71405057e-01 5.27224286e-01 2.97096290e-02 6.44317884e-03
  1.93506883e-01 2.59136872e-04 6.96940604e-02]
 [5.48494007e-02 1.85211904e-02 8.45714429e-01 7.00995760e-02
  9.13570939e-03 1.36375032e-05 2.91921252e-03]
 [6.32391098e-03 3.70646254e-03 6.46849359e-02 9.16507407e-01
  1.45157137e-03 1.84403261e-06 4.89964263e-04]
 [1.26748443e-01 1.97189836e-01 1.49334007e-02 2.57138481e-03
  5.17377214e-01 6.56929376e-04 1.64906503e-01]
 [1.06145871e-04 1.37095457e-04 1.15732957e-05 1.69591086e-06
  3.41055002e-04 9.96555653e-01 2.35085442e-03]
 [2.72375740e-02 4.60024409e-02 3.09085872e-03 5.62199021e-04
  1.06815624e-01 2.93303330e-03 7.98748948e-01]]
[1.0058507  0.99824223 1.00125316 0.9931661  1.02438371 0.99950407
 0.98539068]


In [66]:
# evaluate test performance using the average log-likelihood as metric
# it is defined as log(Pr(d_test))/n_test)
# n_test number of transition in the test set
songs = len(song_hash)
n_test = np.sum(pg.transition_count(songs,test_dataset))

#result = sum( sum(dimension.D[s, i.index()] ) for i in range(len(test_dataset)))  